# 1. 환경 설정 및 필수 라이브러리 설치

In [2]:
# evaluation.ipynb - CELL 1 (최종 안정화 코드)

# 1. 필수 라이브러리 설치 (버전 충돌 방지 및 KoBERT 로드를 위해 최신 환경 사용)
!pip install transformers
!pip install torch # Colab이 현재 지원하는 최신 버전을 설치합니다.
!pip install seqeval pytorch-crf tqdm

# 2. KoBERT 모델 로드를 위한 추가 라이브러리 설치 (Hugging Face의 AutoModel을 사용하기 위해)
!pip install sentencepiece
!pip install tokenizers

# 3. 한글 폰트 설정
!sudo apt-get install -y fonts-nanum > /dev/null
!sudo fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib

print("라이브러리 설치 완료.")

라이브러리 설치 완료.


# 2. 모델 구조 정의 및 데이터 준비

In [1]:
# evaluation.ipynb - CELL 2 (런타임 재시작 후 실행)

import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
# 👇 KoBERT 로드를 위한 임포트를 AutoTokenizer/AutoModel로 대체 (설치 실패 우회)
from transformers import AutoTokenizer, AutoModel, BertModel
from torch.nn.utils.rnn import pad_sequence
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm
from torch import nn
from torchcrf import CRF

# ----------------- A. 상수 및 경로 정의 -----------------
KOBERT_MODEL_NAME = "skt/kobert-base-v1"
labels = ['O', 'B-SUB', 'I-SUB', 'B-DT', 'I-DT', 'B-LOC', 'I-LOC']
label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for i, label in enumerate(labels)}
NUM_LABELS = len(labels)
MAX_LEN = 64
RANDOM_SEED = 42 # 데이터 분할의 재현성을 위해 시드 고정

# 👇 Google Drive 마운트
from google.colab import drive
if not os.path.exists('/content/drive'):
    print("🔄 Google Drive를 마운트합니다...")
    drive.mount('/content/drive')
else:
    print("✅ Google Drive가 이미 마운트되어 있습니다.")

# 👇 Drive 경로 설정
ML_ASSIGNMENT5_PATH = '/content/drive/MyDrive/ML_assignment5/'

# 1. 모델 가중치 경로 (확인된 경로)
BEST_MODEL_PATH = os.path.join(ML_ASSIGNMENT5_PATH, 'best_kobert_crf_model.pt')

# 2. 데이터 파일 경로 (데이터 파일이 Drive에 있다고 가정)
DATA_PATH = os.path.join(ML_ASSIGNMENT5_PATH, 'slack_calendar_data.csv')


# ----------------- B. 데이터 로드 및 전처리 재현 -----------------
try:
    df = pd.read_csv(DATA_PATH, encoding='utf-8')
    df['location'] = df['location'].fillna('-')
except FileNotFoundError:
    print(f"❌ 오류: 데이터 파일을 찾을 수 없습니다. 경로를 확인해주세요: {DATA_PATH}")
    exit()

tokenizer = AutoTokenizer.from_pretrained(KOBERT_MODEL_NAME)

# [get_bio_tags 함수 정의] - training.ipynb와 동일한 함수 코드를 그대로 복사합니다.
def get_bio_tags(message, title, location):
    tokens = tokenizer.tokenize(message)
    tags = ['O'] * len(tokens)

    entity_map = {}
    if title != '-': entity_map['SUB'] = tokenizer.tokenize(title)
    if location != '-': entity_map['LOC'] = tokenizer.tokenize(location)

    for entity_tag, entity_tokens in entity_map.items():
        if not entity_tokens: continue
        for i in range(len(tokens) - len(entity_tokens) + 1):
            if tokens[i:i+len(entity_tokens)] == entity_tokens:
                tags[i] = f'B-{entity_tag}'
                for j in range(1, len(entity_tokens)):
                    tags[i+j] = f'I-{entity_tag}'
                break

    time_keywords = ['시', '분', '월', '일', '주', '내일', '모레', '오늘', '오전', '오후', '저녁']
    for i, token in enumerate(tokens):
        if tags[i] == 'O':
            if any(keyword in token for keyword in time_keywords):
                tags[i] = 'B-DT'

    if len(tokens) != len(tags):
        tags = ['O'] * len(tokens)

    return tokens, [label_to_id[tag] for tag in tags]

# [데이터셋 변환 및 분할 (Test Set 추출)] - training.ipynb와 동일한 로직 사용
tokenized_data = []
for index, row in df.iterrows():
    tokens, tag_ids = get_bio_tags(row['message'], row['title'], row['location'])
    input_ids = tokenizer.convert_tokens_to_ids(['[CLS]'] + tokens + ['[SEP]'])
    tag_ids = [label_to_id['O']] + tag_ids + [label_to_id['O']]

    if len(input_ids) > MAX_LEN:
        input_ids = input_ids[:MAX_LEN-1] + [input_ids[-1]]
        tag_ids = tag_ids[:MAX_LEN-1] + [tag_ids[-1]]

    tokenized_data.append({'input_ids': torch.tensor(input_ids), 'labels': torch.tensor(tag_ids)})

# Train/Validation/Test 분할 (랜덤 시드=42로 동일하게 분할 재현)
train_data, temp_data = train_test_split(tokenized_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Test Set 크기: {len(test_data)}개")

✅ Google Drive가 이미 마운트되어 있습니다.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Test Set 크기: 100개


# 모델 클래스 정의 및 가중치 로드

In [2]:
# evaluation.ipynb - CELL 3

# [Dataset 및 Collate 함수 정의] - training.ipynb의 Cell 3 코드를 그대로 복사합니다.
class ScheduleDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    # Padding을 위한 함수 (KoBERT의 [PAD] 토큰 ID: 1)

    # input_ids와 labels를 분리
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]

    # 패딩 마스크 생성 (0이 패딩 위치)
    attention_mask = [torch.ones_like(ids) for ids in input_ids]

    # 패딩 적용
    # [주의] tokenizer.vocab['[PAD]']는 KoBERT 설치 방식에 따라 오류가 날 수 있으므로, 임시로 1(PAD ID) 사용
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=1)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=label_to_id['O'])
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)

    return {'input_ids': input_ids_padded, 'labels': labels_padded, 'attention_mask': attention_mask_padded}

# [KoBERT-CRF 모델 클래스 정의] - training.ipynb의 Cell 4 코드를 그대로 복사합니다.
class KoBERTNerModel(nn.Module):
    def __init__(self, num_labels):
        super(KoBERTNerModel, self).__init__()
        # KOBERT_MODEL_NAME은 Cell 2에서 정의됨
        self.bert = AutoModel.from_pretrained(KOBERT_MODEL_NAME)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            mask = attention_mask.bool()
            loss = -self.crf(logits, labels, mask=mask, reduction='mean')
            return loss, logits
        else:
            mask = attention_mask.bool()
            tags = self.crf.decode(logits, mask=mask)
            return tags, logits

# Test DataLoader 및 모델 인스턴스화
BATCH_SIZE = 32
# test_data는 Cell 2에서 이미 분할되었습니다.
test_dataloader = DataLoader(ScheduleDataset(test_data), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KoBERTNerModel(NUM_LABELS).to(device)

# 저장된 가중치 로드
try:
    # BEST_MODEL_PATH는 Cell 2에서 Google Drive 경로로 설정되었습니다.
    model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
    print(f"✅ 모델 가중치 로드 성공: {BEST_MODEL_PATH}")
except Exception as e:
    print(f"❌ 오류: 모델 가중치 로드 실패. '{BEST_MODEL_PATH}' 경로를 확인하세요.")
    print(f"상세 오류: {e}")
    # 가중치 로드 실패 시 강제 종료
    raise RuntimeError("모델 가중치 로드에 실패했습니다. 경로와 파일 접근 권한을 확인해주세요.")

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

✅ 모델 가중치 로드 성공: /content/drive/MyDrive/ML_assignment5/best_kobert_crf_model.pt


# 평가 지표 정의 및 Test Set 최종 평가

In [3]:
# evaluation.ipynb - CELL 4

# [평가 지표 정의] - Macro F1 Score 계산 및 Classification Report 생성
# 이 함수는 BIO 태그에 맞게 예측 결과와 실제 레이블을 매핑하여 seqeval 리포트를 생성합니다.
def calculate_metrics_for_report(predictions, true_labels, attention_mask):
    true_tags = []
    pred_tags = []

    for pred, label, mask in zip(predictions, true_labels, attention_mask):
        true_sentence_tags = []
        pred_sentence_tags = []

        for p, l, m in zip(pred, label, mask):
            # Attention Mask가 1인 유효 토큰만 평가에 포함
            if m.item() == 1:
                true_tags_str = id_to_label[l.item()]
                pred_tags_str = id_to_label[p]

                true_sentence_tags.append(true_tags_str)
                pred_sentence_tags.append(pred_tags_str)

        if true_sentence_tags:
            true_tags.append(true_sentence_tags)
            pred_tags.append(pred_sentence_tags)

    # Macro F1 Score 계산 (문제 유형에 맞는 지표)
    f1_macro = f1_score(true_tags, pred_tags, average='macro', zero_division=0)

    # Classification Report (태그별 Precision, Recall, F1)
    report = classification_report(true_tags, pred_tags, digits=4, zero_division=0)

    return f1_macro, report

# --- Test Set 최종 평가 시작 ---
model.eval()
all_predictions = []
all_true_labels = []
all_masks = []

print("\n--- Test Set 최종 평가 시작 ---")

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device) # 실제 레이블도 필요

        # CRF 디코딩을 통해 예측된 태그 시퀀스 획득
        predictions, _ = model(input_ids, attention_mask, labels=None)

        # 결과를 CPU로 옮겨 저장
        for pred_seq, true_seq, mask_seq in zip(predictions, labels, attention_mask):
            all_predictions.append(pred_seq)
            all_true_labels.append(true_seq.cpu())
            all_masks.append(mask_seq.cpu())

# 4. 최종 결과 출력
final_f1_macro, final_report = calculate_metrics_for_report(all_predictions, all_true_labels, all_masks)

print("\n\n--- 최종 평가 결과 ---")
print(f"**Test Set Macro F1 Score:** {final_f1_macro:.4f}")
print("\n**Tag별 Precision, Recall, F1 Score:**")
print(final_report)



--- Test Set 최종 평가 시작 ---


Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.13it/s]




--- 최종 평가 결과 ---
**Test Set Macro F1 Score:** 1.0000

**Tag별 Precision, Recall, F1 Score:**
              precision    recall  f1-score   support

         LOC     1.0000    1.0000    1.0000        18
         SUB     1.0000    1.0000    1.0000        75

   micro avg     1.0000    1.0000    1.0000        93
   macro avg     1.0000    1.0000    1.0000        93
weighted avg     1.0000    1.0000    1.0000        93

